In [1]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5

Traceback (most recent call last):
  File "C:\src\qudi\logic\jupyterkernel\qzmqkernel.py", line 679, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-dd746ab76dde>", line 4, in <module>
    awg.awg._ch[1].sample_marker_low = 0
AttributeError: module 'hardware.awg.AWG_M8190A_qudi' has no attribute 'awg'


In [6]:
awg.awg._ch[1].sync_marker_high=1.5

In [7]:
awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5


seq = mcas.MultiChSeq('Laser', ch_dict = {'awg':[1,2]})
seq.awg =awg
seq.start_new_segment('Starting segment')


seq.asc(length_mus = 10, green=True, orange = True)


seq.sequences['awg'][1].print_info()

0     Laser             10.000000
   0     Starting segment  10.000000
      0                       10.000000wait    1       0       
      1     _missing_smpls_   0.000000wait    0       0       


In [8]:
seq.write_seq()

MCAS 'Laser' has been generated.


In [9]:
seq.run_sequence()

writing sequence Laser on AWG_M8190A_qudi ch 1 took 0.0260 seconds
writing sequence Laser on AWG_M8190A_qudi ch 2 took 0.0220 seconds


In [591]:
seq.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0130 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0090 seconds


## Nuclear ODMR

In [9]:
AWG_M8190A_qudi.awg._ch[1].output_amplitude

0.5

In [99]:
awg = AWG_M8190A_qudi
import hardware.awg.multi_channel_awg_seq as mcas

awg.awg._ch[1].sample_marker_low = 0
awg.awg._ch[1].sample_marker_high = 1.5
AWG_M8190A_qudi.awg._ch[1].output_amplitude = 0.5


def awg_nitrogen_odmr(rf_freqs):
    seq = mcas.MultiChSeq('nitrogen_odmr', ch_dict={'awg': [1, 2]})
    awg_freq = 2203.955 # in MHz
    electron_amp = 0.2
    electron_pi = 3.520/2 # us

    
    for freq in rf_freqs:
        
        seq.start_new_segment('nitrogen_odmr')
        seq.asc(length_mus = 1, green=True, orange = True)
        
        seq = awg_AddWait(seq, 1)
        
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
                length_mus=electron_pi)
        
        seq = awg_AddWait(seq, 1)
        
        seq.asc(pdawg1=dict(type='sine', frequencies=[freq], amplitudes=[2*electron_amp]),
                length_mus=200)        
        
        seq = awg_AddWait(seq, 1)
        
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
                length_mus=electron_pi)    
        
        seq = awg_AddWait(seq, 1)
        seq = awg_AddReadout(seq)
        seq = awg_AddWait(seq, 350)
#         ===================
        seq.asc(length_mus = 1, green=True, orange = True)
        seq = awg_AddWait(seq, 1)
        seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
                length_mus=electron_pi)
        seq = awg_AddWait(seq, 1)
        
#         seq.asc(pdawg1=dict(type='sine', frequencies=[freq], amplitudes=[2*electron_amp]),
#                 length_mus=100)        
        
        seq = awg_AddWait(seq, 1)
#         seq.asc(pdawg1=dict(type='sine', frequencies=[awg_freq], amplitudes=[electron_amp]),
#                 length_mus=electron_pi)        
        seq = awg_AddWait(seq, 1)
        seq = awg_AddReadout(seq)
        seq = awg_AddWait(seq, 100)

    return seq





def awg_AddReadout(seq):
    length_laserreadout_ns = 3000
    length_lasersafety_ns = 4000
    AOMdelay_ns = 400
    length_laserreadout_smpl = length_laserreadout_ns / 1e3 * 12e3
    AOMdelay_sample = AOMdelay_ns  / 1e3 * 12e3
    
    while length_laserreadout_smpl % 64 != 0:
        length_laserreadout_smpl += 1
        
    seq.asc(length_mus=0.5)  # 2nd step
#     seq.asc(length_mus = 10, green=False, orange = True) #green on, AOM off

    seq.asc(timetagger=False,green=True,orange=False, length_smpl=AOMdelay_sample)
    seq.asc(timetagger=True,green=True,orange=True, length_smpl=length_laserreadout_smpl)    
    seq.asc(timetagger=True,green=False,orange=True, length_smpl=AOMdelay_sample)

    seq.asc(length_mus= length_lasersafety_ns / 1e3)
    return seq

def awg_AddWait(seq, time, orange=False, green=False):
    enable_reuse_segments = False
    if time < 0.:
        for i in range(3): 
            print ('Negative time for sequence in AWG!!!')
    elif time > 10.:
        if int(time / (32 / 12.)) >= 1:
            seq.start_new_segment('wait_loop', reuse_segment=enable_reuse_segments,
                                  loop_count=int(time / (32 / 12.)))
            seq.asc(length_smpl=32000, orange=orange, green=green)
        seq.start_new_segment('wait_additional')
        seq.asc(length_mus=time % (32 / 12.), orange=orange, green=green)
    else:
        seq.asc(length_mus=time, orange=orange, green=green)
    return seq

def AddRfSafety(seq):
    rf_safety = 1
    seq.asc(length_mus=rf_safety)
    return seq

rf_freqs = np.arange(7.00, 7.25, 0.001)

seqNitrogenOdmr = awg_nitrogen_odmr(rf_freqs)
seqNitrogenOdmr.awg = awg


In [100]:

pulsedmasterlogic.set_measurement_settings(invoke_settings=False, 
                                           controlled_variable= rf_freqs,
                                           number_of_lasers=2*len(rf_freqs), 
                                           laser_ignore_list=[], 
                                           alternating=True, 
                                           units=('MHz', 'arb. u.'))

In [101]:
seqNitrogenOdmr.write_seq()

MCAS 'nitrogen_odmr' has been generated.


In [102]:
seqNitrogenOdmr.run_sequence()

writing sequence nitrogen_odmr on AWG_M8190A_qudi ch 1 took 53.9894 seconds
writing sequence nitrogen_odmr on AWG_M8190A_qudi ch 2 took 35.9347 seconds


In [104]:
seqNitrogenOdmr.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0190 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0200 seconds


In [105]:
seq.run_sequence()

writing sequence Laser on AWG_M8190A_qudi ch 1 took 0.0180 seconds
writing sequence Laser on AWG_M8190A_qudi ch 2 took 0.0180 seconds


In [19]:
seq.stop_awgs()

writing sequence wait on AWG_M8190A_qudi ch 1 took 0.0120 seconds
writing sequence wait on AWG_M8190A_qudi ch 2 took 0.0150 seconds
